# 2 Attention Mechanisms

# 2.1 A Simple Self-Attention Mechanism Without Trainable Weights

In [2]:
import torch

inputs= torch.tensor(
    [[0.43,0.20,0.90],
     [0.56,0.84,0.56],
     [0.22,0.85,0.34],
     [0.77,0.25,0.36],
     [0.05,0.80,0.79],
     [0.77,0.46,0.12]]
)   

In [3]:
input_query= inputs[1]

In [4]:
input_1= inputs[0]

In [5]:
res =0
i=0
res = torch.dot(inputs[i],input_query)
print(res)   

tensor(0.9128)


In [6]:
query=inputs[1]

atten_scores_2 = torch.empty(inputs.shape[0])

for i,x_i in enumerate(inputs):
    atten_scores_2[i]=torch.dot(x_i,query)

print(atten_scores_2)

tensor([0.9128, 1.3328, 1.0276, 0.8428, 1.1424, 0.8848])


In [7]:
attn_weights_2_tmp= atten_scores_2 / atten_scores_2.sum()

attn_weights_2_tmp

tensor([0.1486, 0.2170, 0.1673, 0.1372, 0.1860, 0.1440])

In [8]:
attn_weights_2_tmp.sum()

tensor(1.)

In [9]:
def softmax_navie(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_navie(atten_scores_2)

tensor([0.1469, 0.2236, 0.1648, 0.1370, 0.1848, 0.1429])

In [10]:
atten_weights_2=torch.softmax(atten_scores_2 ,dim=0)

In [11]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):

    print(f"{atten_weights_2[i]} ----> {inputs[i]}")
    context_vec_2+= atten_weights_2[i]*x_i

print(context_vec_2)

0.1469186246395111 ----> tensor([0.4300, 0.2000, 0.9000])
0.2236044555902481 ----> tensor([0.5600, 0.8400, 0.5600])
0.16479112207889557 ----> tensor([0.2200, 0.8500, 0.3400])
0.13698600232601166 ----> tensor([0.7700, 0.2500, 0.3600])
0.18483781814575195 ----> tensor([0.0500, 0.8000, 0.7900])
0.14286194741725922 ----> tensor([0.7700, 0.4600, 0.1200])
tensor([0.4494, 0.6051, 0.5260])


In [12]:
for i,x_i in enumerate(inputs):
    print(i, inputs[i])  # i_x is the index, i is the tensor row


0 tensor([0.4300, 0.2000, 0.9000])
1 tensor([0.5600, 0.8400, 0.5600])
2 tensor([0.2200, 0.8500, 0.3400])
3 tensor([0.7700, 0.2500, 0.3600])
4 tensor([0.0500, 0.8000, 0.7900])
5 tensor([0.7700, 0.4600, 0.1200])


# 2.2 Computing attention weights for all input tokens

In [13]:
attn_scores = torch.empty(6,6)

for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
     attn_scores[i,j]= torch.dot(x_i,x_j)

print(attn_scores)

tensor([[1.0349, 0.9128, 0.5706, 0.7051, 0.8925, 0.5311],
        [0.9128, 1.3328, 1.0276, 0.8428, 1.1424, 0.8848],
        [0.5706, 1.0276, 0.8865, 0.5043, 0.9596, 0.6012],
        [0.7051, 0.8428, 0.5043, 0.7850, 0.5229, 0.7511],
        [0.8925, 1.1424, 0.9596, 0.5229, 1.2666, 0.5013],
        [0.5311, 0.8848, 0.6012, 0.7511, 0.5013, 0.8189]])


In [14]:
attn_weights = torch.softmax(attn_scores,dim=1)

In [15]:
attn_weights

tensor([[0.2126, 0.1881, 0.1336, 0.1529, 0.1844, 0.1284],
        [0.1469, 0.2236, 0.1648, 0.1370, 0.1848, 0.1429],
        [0.1353, 0.2136, 0.1855, 0.1266, 0.1996, 0.1395],
        [0.1687, 0.1935, 0.1380, 0.1827, 0.1406, 0.1766],
        [0.1620, 0.2079, 0.1732, 0.1119, 0.2354, 0.1095],
        [0.1419, 0.2021, 0.1522, 0.1768, 0.1377, 0.1892]])

# 2.3 Implementing self-attention with trainable weights

# 2.3.1 Computing the attention weights step by step

In [16]:
x_2 = inputs[1]

d_in= inputs.shape[1]

d_out=2

In [17]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in ,d_out))
W_key = torch.nn.Parameter(torch.rand(d_in,d_out))
W_value = torch.nn.Parameter(torch.rand(d_in,d_out))

In [18]:
query_2 = x_2 @ W_query

query_2

tensor([0.4187, 1.3529], grad_fn=<SqueezeBackward4>)

In [19]:
keys= inputs @W_key
value=inputs @W_value

keys.shape

torch.Size([6, 2])

In [20]:
keys_2 =keys[1]

attn_score_22=torch.dot(query_2 ,keys_2)

In [21]:
attn_score_22

tensor(1.5944, grad_fn=<DotBackward0>)

In [22]:
attn_scores_2= query_2 @ keys.T

attn_scores_2

tensor([1.2516, 1.5944, 1.3049, 0.7979, 1.6963, 0.7657],
       grad_fn=<SqueezeBackward4>)

In [23]:
d_k=keys.shape[1]

attn_weights_2=torch.softmax(attn_scores_2/d_k**0.5,dim=-1)
attn_weights_2

tensor([0.1635, 0.2083, 0.1698, 0.1186, 0.2239, 0.1159],
       grad_fn=<SoftmaxBackward0>)

In [24]:
context_vec_2 = attn_weights_2 @ value

context_vec_2

tensor([0.2916, 0.7877], grad_fn=<SqueezeBackward4>)

# 2.4 Implementing a compact SelfAttention Class

In [25]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self,d_in,d_out):
     super().__init__()
     self.W_query = torch.nn.Parameter(torch.rand(d_in ,d_out))
     self.W_key = torch.nn.Parameter(torch.rand(d_in,d_out))
     self.W_value = torch.nn.Parameter(torch.rand(d_in,d_out))

    def forward(self,x):
        queries=inputs @ W_query
        keys=inputs @W_key
        values=inputs @W_value

        attn_scores=queries @ keys.T
        attn_weighs= torch.softmax(attn_scores / d_k**0.5,dim=-1)
        context_vec = attn_weights @ values

        
        return context_vec

torch.manual_seed(123)
sa_v1 =SelfAttention_v1(d_in,d_out)
sa_v1(inputs)

tensor([[0.2772, 0.7753],
        [0.2878, 0.7666],
        [0.2910, 0.7680],
        [0.2723, 0.7374],
        [0.2937, 0.7932],
        [0.2754, 0.7298]], grad_fn=<MmBackward0>)

In [26]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):

    def __init__(self,d_in,d_out,qkv_bias=False):
     super().__init__()
     self.W_query = torch.nn.Linear(d_in ,d_out,bias=qkv_bias)
     self.W_key = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
     self.W_value = torch.nn.Linear(d_in,d_out,bias=qkv_bias)

    def forward(self,x):
        queries=self.W_query(inputs)
        keys=self.W_key(inputs)
        values=self.W_value(inputs)

        attn_scores=queries @ keys.T
        attn_weighs= torch.softmax(attn_scores / d_k**0.5,dim=-1)
        context_vec = attn_weights @ values

        
        return context_vec

torch.manual_seed(123)
sa_v2 =SelfAttention_v2(d_in,d_out)
sa_v2(inputs)

tensor([[-0.5356, -0.0864],
        [-0.5437, -0.1187],
        [-0.5406, -0.1238],
        [-0.5433, -0.1087],
        [-0.5355, -0.1077],
        [-0.5460, -0.1225]], grad_fn=<MmBackward0>)

# 2.5 Hiding future words with casual attention 

# Applying a casual attention mask

In [27]:
queries=sa_v2.W_query(inputs)
keys=sa_v2.W_key(inputs)
values=sa_v2.W_value(inputs)

attn_scores=queries @ keys.T
attn_weighs= torch.softmax(attn_scores / d_k**0.5,dim=-1)

In [28]:
context_length = attn_scores.shape[0]

mask_simple =torch.tril(torch.ones(context_length, context_length))
                        
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [29]:
masked_simple =attn_weights * mask_simple
masked_simple

tensor([[0.2126, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1469, 0.2236, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1353, 0.2136, 0.1855, 0.0000, 0.0000, 0.0000],
        [0.1687, 0.1935, 0.1380, 0.1827, 0.0000, 0.0000],
        [0.1620, 0.2079, 0.1732, 0.1119, 0.2354, 0.0000],
        [0.1419, 0.2021, 0.1522, 0.1768, 0.1377, 0.1892]])

In [30]:
row_sums =masked_simple.sum(dim=-1 ,keepdim=True)
masked_simple_norm =masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3965, 0.6035, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2531, 0.3997, 0.3471, 0.0000, 0.0000, 0.0000],
        [0.2470, 0.2834, 0.2020, 0.2675, 0.0000, 0.0000],
        [0.1819, 0.2335, 0.1945, 0.1257, 0.2644, 0.0000],
        [0.1419, 0.2021, 0.1522, 0.1768, 0.1377, 0.1892]])


In [31]:
mask = torch.triu(torch.ones(context_length,context_length,context_length),diagonal=1)
masked= attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[[0.3129,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.1516, 0.2232,   -inf,   -inf,   -inf,   -inf],
         [0.0201, 0.0898, 0.0921,   -inf,   -inf,   -inf],
         [0.2201, 0.2481, 0.1597, 0.2037,   -inf,   -inf],
         [0.1212, 0.1797, 0.1377, 0.1160, 0.1502,   -inf],
         [0.1222, 0.1700, 0.1259, 0.1160, 0.1398, 0.1186]],

        [[0.3129,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.1516, 0.2232,   -inf,   -inf,   -inf,   -inf],
         [0.0201, 0.0898, 0.0921,   -inf,   -inf,   -inf],
         [0.2201, 0.2481, 0.1597, 0.2037,   -inf,   -inf],
         [0.1212, 0.1797, 0.1377, 0.1160, 0.1502,   -inf],
         [0.1222, 0.1700, 0.1259, 0.1160, 0.1398, 0.1186]],

        [[0.3129,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.1516, 0.2232,   -inf,   -inf,   -inf,   -inf],
         [0.0201, 0.0898, 0.0921,   -inf,   -inf,   -inf],
         [0.2201, 0.2481, 0.1597, 0.2037,   -inf,   -inf],
         [0.1212, 0.1797, 0.1377, 0.1160, 0.1502,   

In [32]:
attn_weights=torch.softmax(masked /keys.shape[-1]**0.5,dim=-1)
print(attn_weights)

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4874, 0.5126, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3223, 0.3386, 0.3391, 0.0000, 0.0000, 0.0000],
         [0.2521, 0.2571, 0.2416, 0.2492, 0.0000, 0.0000],
         [0.1972, 0.2055, 0.1995, 0.1965, 0.2013, 0.0000],
         [0.1655, 0.1712, 0.1659, 0.1648, 0.1676, 0.1651]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4874, 0.5126, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3223, 0.3386, 0.3391, 0.0000, 0.0000, 0.0000],
         [0.2521, 0.2571, 0.2416, 0.2492, 0.0000, 0.0000],
         [0.1972, 0.2055, 0.1995, 0.1965, 0.2013, 0.0000],
         [0.1655, 0.1712, 0.1659, 0.1648, 0.1676, 0.1651]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4874, 0.5126, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3223, 0.3386, 0.3391, 0.0000, 0.0000, 0.0000],
         [0.2521, 0.2571, 0.2416, 0.2492, 0.0000, 0.0000],
         [0.1972, 0.2055, 0.1995, 0.1965, 0.2013, 0.

# Masking additional attention weights with dropout

In [34]:
torch.manual_seed(123)

layer=torch.nn.Dropout(0.5)


In [35]:
drop_rate=0.3

1/(1-drop_rate)

1.4285714285714286

In [36]:
layer(attn_weights)

tensor([[[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6446, 0.6771, 0.6783, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5143, 0.4831, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.4111, 0.0000, 0.3930, 0.0000, 0.0000],
         [0.0000, 0.3424, 0.3318, 0.3295, 0.3351, 0.0000]],

        [[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.6771, 0.6783, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.4831, 0.4984, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.4026, 0.0000],
         [0.3310, 0.3424, 0.0000, 0.3295, 0.3351, 0.3301]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9747, 1.0253, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5143, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3990, 0.3930, 0.0000, 0.

# Implementing a compact casual self_attention class

In [38]:
batch = torch.stack((inputs,inputs),dim=0)

In [44]:
import torch
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.0, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)

        # Causal mask (upper triangular)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], float('-inf')
        )

        attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values

        return context_vec

torch.manual_seed(123)

batch = torch.randn(2, 6, 3)  # [batch_size=2, seq_len=6, d_in=3]
d_in = 3
d_out = 4
context_length = batch.shape[1]
dropout = 0.0

ca = CausalAttention(d_in, d_out, context_length, dropout)
out = ca(batch)


# 2.6 Extending single-head attention to multi-head attention

# 2.6.1 Stacking multiple single-head attention layers

In [55]:
import torch
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.0, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], float('-inf'))
        attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec


class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.0, qkv_bias=False, num_heads=2):
        super().__init__()
        self.heads = nn.ModuleList([
            CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
            for _ in range(num_heads)
        ])

    def forward(self, x):
        # Concatenate outputs from each head along feature dimension
        return torch.cat([head(x) for head in self.heads], dim=-1)

torch.manual_seed(123)

batch = torch.randn(2, 6, 3)  # [batch_size, seq_len, d_in]
context_length = batch.shape[1]
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, qkv_bias=False, num_heads=2)
mha(batch)


tensor([[[-0.0408, -0.0034,  0.1444, -0.1567],
         [-0.0025,  0.0059, -0.0884,  0.0969],
         [-0.0225,  0.1421, -0.1455,  0.2462],
         [ 0.1109, -0.0607, -0.0348,  0.0332],
         [ 0.0667, -0.2384, -0.0740, -0.0419],
         [ 0.0708, -0.2885,  0.0132, -0.1225]],

        [[ 0.1124, -0.6109,  0.0540, -0.4721],
         [ 0.2313, -0.2813,  0.1472, -0.3270],
         [ 0.1189, -0.3333,  0.2083, -0.3498],
         [ 0.1311, -0.2280,  0.0920, -0.2821],
         [ 0.1639, -0.1306,  0.0992, -0.1320],
         [ 0.1950, -0.0952,  0.1747, -0.1882]]], grad_fn=<CatBackward0>)

# 2.6.2 Implementing multi-head attention with weight splits

In [56]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, hea
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head
        
        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [58]:
import torch

torch.manual_seed(123)

batch_size = 2
max_length = 6        # sequence length
output_dim = 3        # embedding dimension
d_in = output_dim
d_out = d_in

# random batch of input embeddings
input_embeddings = torch.randn(batch_size, max_length, d_in)

mha = MultiHeadAttentionWrapper(d_in, d_out, max_length, dropout=0.0, qkv_bias=False, num_heads=2)
context_vecs = mha(input_embeddings)

print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 6])
